In [81]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LassoCV
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import ElasticNetCV

In [164]:
data = pd.read_pickle("MEMV-1.pkl")
data

,EMV,Political Uncertainty Tracker,Infectious Disease EMV Tracker,Macroeconomic News and Outlook EMV Tracker,Macro – Broad Quantity Indicators EMV Tracker,Macro – Inflation EMV Indicator,Macro – Interest Rates EMV Tracker,Macro – Other Financial Indicators EMV Tracker,Macro – Labor Markets EMV Tracker,Macro – Real Estate Markets EMV Tracker,...,National Security Policy EMV Tracker,Government-Sponsored Enterprises EMV Tracker,Trade Policy EMV Tracker,Healthcare Policy EMV Tracker,Food and Drug Policy EMV Tracker,"Transportation, Infrastructure, and Public Utilities EMV Tracker",Elections and Political Governance EMV Tracker,Agricultural Policy EMV Tracker,Petroleum Markets EMV Tracker,VIX
Date,,,,,,,,,,,,,,,,,,,,,
1990-01,17.8882,7.9697,0.2411,13.7416,4.5323,6.7020,6.5574,0.6268,3.1823,5.4484,...,2.0251,0.6268,0.4339,0.3857,0.1929,0.3375,0.5304,0.0482,3.567993,21.990000
1990-02,22.1500,9.6857,0.7699,16.4644,3.7312,8.5876,8.5283,0.9476,3.6127,6.3963,...,1.7175,0.8291,0.3553,0.3553,0.1777,0.1777,0.4146,0.0000,4.441847,19.730000
1990-03,17.6410,8.7400,0.2673,12.0279,3.1184,5.6576,5.2121,0.3118,2.3610,4.5884,...,1.6928,0.6682,0.4009,0.2227,0.0891,0.2227,0.3564,0.0000,3.073809,19.520000
1990-04,15.9283,6.8080,0.1493,11.3987,2.8870,4.0816,4.5794,0.4978,2.9866,5.5749,...,1.0453,0.7466,0.3484,0.2489,0.1493,0.0996,0.2987,0.0000,2.837237,17.370001
1990-05,12.1001,6.0545,0.2521,9.2011,2.6469,4.1594,4.4955,0.6302,2.7309,3.9913,...,0.8823,0.6722,0.3781,0.2941,0.3361,0.0420,0.2521,0.0420,1.554521,15.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-06,21.9521,10.0916,12.5927,14.2944,3.4034,6.1262,4.4245,0.8509,6.2963,5.9560,...,2.2122,0.6807,0.1702,2.3824,0.5105,0.3403,1.1912,0.0000,4.254290,18.240000
2021-07,16.6235,7.2485,10.0712,11.6486,3.5188,4.6109,2.7908,0.2427,4.6109,3.1548,...,1.8201,0.2427,0.0000,0.7280,0.0000,0.4854,0.2427,0.0000,2.669466,16.480000
2021-08,19.6411,9.0514,11.0266,12.4049,3.2735,4.3073,2.2398,0.6892,5.5133,3.7904,...,1.2060,0.1723,0.3446,1.2060,0.5169,0.1723,0.5169,0.0000,4.479545,23.139999


In [31]:
X = np.array(data.iloc[:, :-1])
y = np.array(data.iloc[:, -1])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42)

In [160]:
## OLS
reg = LinearRegression().fit(X_train, y_train)
print(X_train.shape)
print(reg.rank_)
print(reg.score(X_train, y_train))
y_pred_reg = reg.predict(X_test)
mean_squared_error(y_test, y_pred_reg, squared=False)

(305, 45)
45
0.6517108062946986


4.998448074350655

In [178]:
## LASSO Regression
lasso_reg = LassoCV(cv=5, random_state=0).fit(X_train, y_train)
# lasso_reg.score(X_train, y_train)
y_pred_lasso = lasso_reg.predict(X_test)
print(mean_squared_error(y_test, y_pred_lasso, squared=False))
print(lasso_reg.score(X_train, y_train))
print(lasso_reg.alpha_)


4.656039469031619
0.5956750027785043
0.1607192893513345


In [177]:
## Ridge Regression
ridge_reg = RidgeCV().fit(X_train, y_train)
y_pred_ridge = ridge_reg.predict(X_test)
print(mean_squared_error(y_test, y_pred_ridge, squared=False))
print(ridge_reg.score(X_train, y_train))
print(ridge_reg.alpha_)

4.697588094499632
0.6355885227394593
10.0


In [193]:
## ElasticNet for Entire Dataset
elastic_net = ElasticNetCV(l1_ratio = [.1, .3, .5, .7, .95, .99, 1], max_iter = 10000, cv=5, random_state=0)
elastic_net.fit(X_train, y_train)
y_pred_elastic = elastic_net.predict(X_test)
print(elastic_net.l1_ratio_)
print(elastic_net.alpha_)
print(elastic_net.score(X_train, y_train))
print(mean_squared_error(y_test, y_pred_elastic, squared = False))
a = np.array(list(np.argsort(np.abs(elastic_net.coef_)))[::-1])[:3]
col = data.columns
top_feature = []
for index in a:
    top_feature.append((col[index], elastic_net.coef_[index]))
print(top_feature)



## Try np.arange(0.80, 1.01, 0.01)

1.0
0.1607192893513345
0.5956750027785043
4.656039469031619
[('Labor Disputes EMV Tracker', 2.3878201338283915), ('Financial Regulation EMV Tracker', 1.2900901657332917), ('Macro – Interest Rates EMV Tracker', -1.0177857385656526)]


In [179]:
elastic_net = ElasticNetCV(l1_ratio = np.round(np.arange(0.80, 1.01, 0.01), 2), max_iter = 1000, cv=5, random_state=0)
elastic_net.fit(X_train, y_train)
y_pred_elastic = elastic_net.predict(X_test)
print(elastic_net.l1_ratio_)
print(elastic_net.alpha_)
print(elastic_net.score(X_train, y_train))
mean_squared_error(y_test, y_pred_elastic, squared = False)

1.0
0.1607192893513345
0.5956750027785043


4.656039469031619

In [74]:
## Data Processing
s1 = data.loc['1990-01':'1993-06']
s2 = data.loc['1993-07':'1998-06']
s3 = data.loc['1998-07':'2002-12']
s4 = data.loc['2003-01':'2007-09']
s5 = data.loc['2007-10':'2009-12']
s6 = data.loc['2012-01':'2019-12']

In [182]:
## ENR for s1
elastic_net = ElasticNetCV(l1_ratio = [.1, .5, .7, .9, .95, .99, 1], max_iter = 10000, cv=5, random_state=0)
elastic_net.fit(s1.iloc[:,0:-1], s1.iloc[:,-1])
print(elastic_net.coef_)
print(elastic_net.alpha_)
print(elastic_net.l1_ratio_)
print(elastic_net.score(s1.iloc[:,0:-1], s1.iloc[:,-1]))
a = np.array(list(np.argsort(np.abs(elastic_net.coef_)))[::-1])[:3]
col = data.columns
top_feature = []
for index in a:
    top_feature.append((col[index], elastic_net.coef_[index]))
print(top_feature)



[-0.31923005  0.13814916 -0.64363876 -1.07935936  0.50166354  0.19832389
  0.56345674  1.50665573 -0.38005263  1.80156769  0.44023762 -0.62276883
  0.62903496  0.31909131 -0.         -0.3113712  -0.40548157  0.08448134
  0.93974838  0.59363736  0.02283582 -0.95072509 -0.52900873  0.60063307
  0.15334379  0.64872831  0.69385637  0.51598176 -1.80266967  0.13459386
  0.4440254   0.          0.85130912  0.53480088  0.09689366 -0.29607868
 -1.15033998  0.08637765 -0.3849758  -0.          0.58411446 -0.06327429
 -1.28130399  0.08914139  1.42143362]
0.0611847115565497
0.1
0.8238877173256866
[('Competition Policy EMV Tracker', -1.8026696668986617), ('Macro – Real Estate Markets EMV Tracker', 1.8015676878458775), ('Macro – Other Financial Indicators EMV Tracker', 1.5066557344864016)]


In [187]:
## ENR for s2
elastic_net2 = ElasticNetCV(l1_ratio = [.1, .5, .7, .9, .95, .99, 1], max_iter = 10000, cv=5, random_state=0)
elastic_net2.fit(s2.iloc[:,0:-1], s2.iloc[:,-1])
print(elastic_net2.coef_)
print(elastic_net2.alpha_)
print(elastic_net2.l1_ratio_)
print(elastic_net2.score(s2.iloc[:,0:-1], s2.iloc[:,-1]))
a = np.array(list(np.argsort(np.abs(elastic_net2.coef_)))[::-1])[:3]
col = data.columns
top_feature = []
for index in a:
    top_feature.append((col[index], elastic_net2.coef_[index]))
print(top_feature)

[ 1.06243409  0.          0.          0.         -0.61752466  0.
 -1.30985226 -0.          0.07420915  0.         -0.          0.
 -0.          0.         -0.04408493 -0.          0.         -0.
 -0.          0.          0.          0.          0.02571265 -0.
  0.          0.          0.          0.          0.         -0.
  0.         -0.          0.         -0.         -0.          0.
  0.         -0.         -0.          0.         -0.          0.
  0.          0.         -0.        ]
0.5262736824872697
1.0
0.5516897741526328
[('Macro – Interest Rates EMV Tracker', -1.3098522586375818), ('EMV', 1.062434090956899), ('Macro – Broad Quantity Indicators EMV Tracker', -0.6175246574473652)]


In [188]:
## ENR for s3
elastic_net3 = ElasticNetCV(l1_ratio = [.1, .5, .7, .9, .95, .99, 1], max_iter = 10000, cv=5, random_state=0)
elastic_net3.fit(s3.iloc[:,0:-1], s3.iloc[:,-1])
print(elastic_net3.coef_)
print(elastic_net3.alpha_)
print(elastic_net3.l1_ratio_)
print(elastic_net3.score(s3.iloc[:,0:-1], s3.iloc[:,-1]))
a = np.array(list(np.argsort(np.abs(elastic_net3.coef_)))[::-1])[:3]
col = data.columns
top_feature = []
for index in a:
    top_feature.append((col[index], elastic_net3.coef_[index]))
print(top_feature)

[ 0.08197029  0.          0.          0.07864379  0.          0.
 -0.00440556  0.          0.          0.06583019  0.          0.
  0.          0.10777241  0.01634354  0.          0.          0.
 -0.          0.04074801  0.          0.12717054  0.08881463  0.
  0.         -0.10829843  0.          0.00067574 -0.         -0.
  0.         -0.          0.         -0.          0.          0.
  0.11651994  0.          0.          0.          0.          0.
  0.         -0.          0.02986594]
9.27132769244696
0.1
0.2696756566581153
[('Fiscal Policy EMV Tracker', 0.12717054016404875), ('National Security Policy EMV Tracker ', 0.1165199424671226), ('Monetary Policy EMV Tracker', -0.10829843151748653), ('Commodity Markets EMV Tracker', 0.10777241094221497), ('Taxes EMV Tracker', 0.08881463350984507), ('EMV', 0.08197029416119934), ('Macroeconomic News and Outlook EMV Tracker', 0.07864379231124216), ('Macro – Real Estate Markets EMV Tracker', 0.06583019483250002), ('Labor Disputes EMV Tracker', 

In [189]:
## ENR for s4
elastic_net4 = ElasticNetCV(l1_ratio = [.1, .5, .7, .9, .95, .99, 1], max_iter = 10000, cv=5, random_state=0)
elastic_net4.fit(s4.iloc[:,0:-1], s4.iloc[:,-1])
print(elastic_net4.coef_)
print(elastic_net4.alpha_)
print(elastic_net4.l1_ratio_)
print(elastic_net4.score(s4.iloc[:,0:-1], s4.iloc[:,-1]))
a = np.array(list(np.argsort(np.abs(elastic_net4.coef_)))[::-1])[:3]
col = data.columns
top_feature = []
for index in a:
    top_feature.append((col[index], elastic_net4.coef_[index]))
print(top_feature)

[ 0.          0.         -0.          0.34625949 -0.          0.
  0.          0.         -0.53825117  0.         -0.          0.
 -0.         -0.         -0.         -0.          0.         -0.
 -0.          0.         -0.          1.49629157  0.          0.
 -0.78203407 -0.35925891 -0.         -0.          0.         -0.
 -0.         -0.          0.         -0.         -0.          0.
  0.22446351  0.         -0.         -0.          0.          0.
 -0.         -0.         -0.24912154]
0.24266668959677037
1.0
0.6564167572251227
[('Fiscal Policy EMV Tracker', 1.4962915694423624), ('Entitlement and Welfare Programs EMV Tracker', -0.7820340697835348), ('Macro – Labor Markets EMV Tracker', -0.5382511686010011)]


In [190]:
## ENR for s5
elastic_net5 = ElasticNetCV(l1_ratio = [.1, .5, .7, .9, .95, .99, 1], max_iter = 10000, cv=5, random_state=0)
elastic_net5.fit(s5.iloc[:,0:-1], s5.iloc[:,-1])
print(elastic_net5.coef_)
print(elastic_net5.alpha_)
print(elastic_net5.l1_ratio_)
print(elastic_net5.score(s5.iloc[:,0:-1], s5.iloc[:,-1]))
a = np.array(list(np.argsort(np.abs(elastic_net5.coef_)))[::-1])[:3]
col = data.columns
top_feature = []
for index in a:
    top_feature.append((col[index], elastic_net5.coef_[index]))
print(top_feature)

[ 0.11794452 -0.         -0.19958028  0.34043062  0.28605992 -0.3432824
 -0.11260718 -0.19517489  0.45722027  0.18505883  0.         -0.
 -0.75628609 -0.38670009  0.60659202 -0.          0.13656169  0.
 -0.02255564  0.          0.07971106  0.60493216  0.41320992  0.20452492
  0.         -1.10730337 -0.13121859 -0.          0.          0.
  0.          0.         -0.          0.         -0.         -0.05301379
  0.47770847  1.0230123   0.13611478  0.00469213  0.01124843  0.08986616
 -0.03110055  0.         -0.18086476]
0.9795741288779833
0.1
0.8297012768186249
[('Monetary Policy EMV Tracker', -1.1073033652543076), ('Government-Sponsored Enterprises EMV Tracker', 1.0230123030775118), ('Macro – Consumer Spending and Sentiment EMV Tracker', -0.7562860932026972)]


In [191]:
 ## ENR for s6
elastic_net6 = ElasticNetCV(l1_ratio = [.1, .5, .7, .9, .95, .99, 1], max_iter = 10000, cv=5, random_state=0)
elastic_net6.fit(s6.iloc[:,0:-1], s6.iloc[:,-1])
print(elastic_net6.coef_)
print(elastic_net6.alpha_)
print(elastic_net6.l1_ratio_)
print(elastic_net6.score(s6.iloc[:,0:-1], s6.iloc[:,-1]))
a = np.array(list(np.argsort(np.abs(elastic_net6.coef_)))[::-1])[:3]
col = data.columns
top_feature = []
for index in a:
    top_feature.append((col[index], elastic_net6.coef_[index]))
print(top_feature)

[ 0.08661955  0.07024828 -0.0292116   0.00724067 -0.2659847  -0.03046387
  0.00261619  0.          0.16221855  0.34508513  0.          0.
  0.          0.06432387  0.06639411  0.         -0.11153231 -0.
  0.          0.          0.         -0.          0.23426357 -0.24098342
 -0.0413268   0.          0.          0.28723129  0.          0.
 -0.         -0.          0.          0.         -0.          0.
  0.         -0.0102245   0.         -0.1218537  -0.          0.
 -0.20612345 -0.          0.2455556 ]
0.9772371069254541
0.1
0.3260353832505847
[('Macro – Real Estate Markets EMV Tracker', 0.34508513378360867), ('Financial Regulation EMV Tracker', 0.28723128706801593), ('Macro – Broad Quantity Indicators EMV Tracker', -0.2659846978347843)]
